In [1]:
%load_ext watermark
# %load_ext rpy2.ipython
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
# import rpy2
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import beta
from scipy.stats import multinomial


import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from myst_nb import glue
from slugify import slugify

from typing import List



# from rpy2.robjects.packages import importr

import plastockconf as psc
import plastock as pstk

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc

pd.set_option('display.max_rows', 100)


In [2]:
mone = pd.read_csv('data/end_pipe/macro_data_msquared.csv')
mone["date"] = pd.to_datetime(mone['date'], format='mixed', dayfirst=True)
mone["date"] = mone["date"].dt.strftime('%Y-%m-%d')
mone["échantillon"] = list(zip(mone.Plage, mone["date"]))
mtwo = pd.read_csv('data/end_pipe/macro_current.csv')

# these needed to be removed
# not_these = [
#     'G9492',
#     'G989',
#     'G9147',
#     'G9135',
#     'G9134',
#     'G9493',
#     'G9145',
#     'G9331',
#     'G9146',
#     'G9491'
#     ]

# mtwo = mtwo[~mtwo.Code.isin(not_these)]
# mtwo.to_csv('data/end_pipe/macro_current.csv', index=False)
mtwo["Date"] = pd.to_datetime(mtwo.Date, format='mixed', dayfirst=True)
mtwo["Date"] = mtwo["Date"].dt.strftime('%Y-%m-%d')


new_names = {'Substrat':'substrat', 'Date':'date', 'Code':'code', 'Aire':'area', 'Quantité':'quantité'}
columns = ['échantillon', 'Plage', 'date', 'area', 'quantité', 'code']

t1 = mone[~mone.isna()][columns]

In [3]:
t1.head()

,échantillon,Plage,date,area,quantité,code
0,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G159
1,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G10
2,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G103
3,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G106
4,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G117


In [4]:
mtwo['échantillon'] = list(zip(mtwo.Plage, mtwo.Date))
mtwo.rename(columns=new_names, inplace=True)
t2 = mtwo[columns]

In [5]:
t2.head()

,échantillon,Plage,date,area,quantité,code
0,"(Amphion, 2022-02-01)",Amphion,2022-02-01,98,2,G24
1,"(Amphion, 2022-02-01)",Amphion,2022-02-01,342,42,G24
2,"(Amphion, 2022-05-03)",Amphion,2022-05-03,342,5,G24
3,"(Amphion, 2022-07-19)",Amphion,2022-07-19,98,5,G24
4,"(Amphion, 2022-07-19)",Amphion,2022-07-19,342,11,G24


In [6]:
t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2147 entries, 0 to 2146
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   échantillon  2147 non-null   object
 1   Plage        2147 non-null   object
 2   date         2147 non-null   object
 3   area         2147 non-null   int64 
 4   quantité     2147 non-null   int64 
 5   code         2147 non-null   object
dtypes: int64(2), object(4)
memory usage: 100.8+ KB


In [7]:
t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7154 entries, 0 to 7153
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   échantillon  7154 non-null   object
 1   Plage        7154 non-null   object
 2   date         7154 non-null   object
 3   area         7154 non-null   int64 
 4   quantité     7154 non-null   int64 
 5   code         7154 non-null   object
dtypes: int64(2), object(4)
memory usage: 335.5+ KB


In [8]:
t1['quantité'].sum()

28107

In [9]:
t2['quantité'].sum()

28107

In [10]:
t2["échantillon"].nunique()

98

In [11]:
t1["échantillon"].nunique()

98

## Unknown codes

In [12]:
mone_codes = mone.code.unique()
mtwo_codes = mtwo.code.unique()
gfrag_gfoam_gcaps = ['G21', 'G23', 'G24', 'G78', 'G79', 'G80', 'G81', 'G82',]
diff_mtwo_mone = set(mtwo_codes).difference(set(mone_codes))

codes_in_mtwo_not_in_mone = [x for x in diff_mtwo_mone if x not in gfrag_gfoam_gcaps]
codes_in_mtwo_not_in_mone

[]

In [13]:
# codes in mtwo not in mone:
mt_wtf = mtwo[mtwo.code.isin(codes_in_mtwo_not_in_mone)]
mt_wtf['quantité'].sum()

0

## Find the errors

C'est possible que il'n'y a pas de distinction entre les deux zones pour revernges le 13 août. 

In [14]:
t2 = t2[~t2.code.isin(codes_in_mtwo_not_in_mone)]

t1_dt = t1.groupby("échantillon")[["quantité"]].sum()
t2_dt = t2.groupby("échantillon")[["quantité"]].sum()

t1_dt['s'] = [str(x) for x in t1_dt.index]
t2_dt['s'] = [str(x) for x in t2_dt.index]

t1_dt.set_index('s', inplace=True)
t2_dt.set_index('s', inplace=True)

### difference

In [15]:
print(f'Il y a {t2_dt["quantité"].sum() - t1_dt["quantité"].sum()} plus d\'objets en t2')

Il y a 0 plus d'objets en t2


In [16]:
t1_dt['t2_q'] = t1_dt.index.map(lambda x: t2_dt.loc[x, 'quantité'])
t1_dt['diff'] = t1_dt['quantité'] - t1_dt.t2_q

t1_dt['diff'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### l'échantillon

In [17]:
t1_dt[t1_dt['diff'] == -614]

,quantité,t2_q,diff
s,,,


In [18]:
t1samp = t1.loc[(t1.Plage=="Préverenges") & (t1["date"] == "2022-08-13")]
t2samp = t2.loc[(t2.Plage=="Préverenges") & (t2["date"] =="2022-08-13")]

In [19]:
mone[(mone.Plage == "Préverenges")&(mone['date'] == "2022-08-13")&(mone['quantité'] > 0)]

,échantillon,Plage,substrat,date,area,slug,code,quantité,pcs/m²,region
6722,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G112,69,0.062106,Grand lac
6723,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G113,5,0.004500,Grand lac
6724,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G125,2,0.001800,Grand lac
6725,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G131,1,0.000900,Grand lac
6726,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G211,4,0.003600,Grand lac
6727,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G26,1,0.000900,Grand lac
6728,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G27,232,0.208821,Grand lac
6729,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G30,28,0.025203,Grand lac
6730,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G31,2,0.001800,Grand lac
6731,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,1111,preverenges,G32,1,0.000900,Grand lac


In [20]:
mtwo[(mtwo.Plage == "Préverenges")&(mtwo['date'] == "2022-08-13")]

,Plage,area,Temps,Position,substrat,date,Saison,Objet,Détail,code,OPSAR,UNEP,quantité,Poids,échantillon
63,Préverenges,961,490,2,NaN,2022-08-13,3,Ballon de baudruche,NaN,G125,49.0,RB01,2,0.001600,"(Préverenges, 2022-08-13)"
127,Préverenges,961,490,2,NaN,2022-08-13,3,Bande élastique,NaN,G131,NaN,NaN,1,0.000280,"(Préverenges, 2022-08-13)"
186,Préverenges,961,490,2,NaN,2022-08-13,3,Bâtonnet de sucette en plastique,NaN,G31,19.0,NaN,2,0.000360,"(Préverenges, 2022-08-13)"
262,Préverenges,961,490,2,NaN,2022-08-13,3,Bouchon/capuchon PET,NaN,G21,15.0,PL01,3,0.005250,"(Préverenges, 2022-08-13)"
348,Préverenges,961,490,2,NaN,2022-08-13,3,Briquet,NaN,G26,16.0,PL10,1,0.012000,"(Préverenges, 2022-08-13)"
510,Préverenges,961,490,2,NaN,2022-08-13,3,Coton-tige,NaN,G95,98.0,OT02,3,0.000540,"(Préverenges, 2022-08-13)"
578,Préverenges,150,80,1,1.0,2022-08-13,3,Couverts,NaN,G34,6.0,PL06,1,0.008000,"(Préverenges, 2022-08-13)"
724,Préverenges,150,80,1,1.0,2022-08-13,3,Emballage (film) à nourriture/alimentaire/en p...,NaN,G30,19.0,NaN,7,0.004100,"(Préverenges, 2022-08-13)"
725,Préverenges,961,490,2,NaN,2022-08-13,3,Emballage (film) à nourriture/alimentaire/en p...,NaN,G30,19.0,NaN,21,0.012200,"(Préverenges, 2022-08-13)"
853,Préverenges,961,490,2,NaN,2022-08-13,3,Ficelle de moins de 1cm de diamètre,NaN,G50,32.0,PL19,19,0.076000,"(Préverenges, 2022-08-13)"
